<a href="https://colab.research.google.com/github/Yuns-u/Boston_Housing_predict_practice/blob/main/N422a_Distributed_Representation_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 4 / SPRINT 2 / NOTE 2*

# 📝 Assignment

---


# Distributed_Representation

## Code

캐글의 [SMS Spam dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) 에 사전 학습된 Word2Vec 임베딩 벡터를 적용하여 분류해봅시다.<br/>
세션 노트에 있었던 단어 임베딩 벡터를 평균내어 분류하는 방법을 적용해봅시다.

In [ ]:
!pip install gensim --upgrade

     |████████████████████████████████| 24.1 MB 2.6 kB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gensim

gensim.__version__

'4.1.2'

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다 `encoding = 'latin-1'` 을 사용합니다.
- 필요없는 열(column)을 삭제합니다.
- LabelEncoder를 사용하여 label 전처리를 해줍니다. 

In [ ]:
from google.colab import files

file = files.upload()

Saving spam.csv to spam.csv


In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
#데이터를 데이터프레임으로 읽어오기
df = pd.read_csv('spam.csv', encoding='latin-1')
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
#불필요한 열 삭제하기: v1, v2를 제외한 칼럼은 불필요하다고 볼 수 있다.
#df = df.drop(columns=['Unnamed: 2','Unnamed: 3','Unnamed: 4'])
#혹은 필요한 열만 남겨서 데이터프레임으로 만들어주기
df = df[['v1','v2']]
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
#shape 확인
df.shape

(5572, 2)

In [ ]:
#ham과 spam 비율
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [18]:
df['v1'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: v1, dtype: float64

In [20]:
#LabelEncoder를 사용하여 label 전처리하기
#spam을 1로, ham을 0으로 처리
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['v1'])
df.head()

,v1,v2,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 15%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words = 1000` 으로 설정합니다.
- pad_sequence의 `maxlen=150` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=64, epochs=10, validation_split=0.2` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [21]:
#데이터셋 분류하기
#test_size의 비율은 15%로, random_state = 42 로 설정하시오.

train, test = train_test_split(df, test_size=0.15, random_state=42)

In [22]:
#label과 학습시킬 데이터 구분하기
X_train = train['v2']
y_train = train['label']

X_test = test['v2']
y_test = test['label']

In [25]:
print(f"Train set shape : {X_train.shape}")
print(f"Test set shape : {X_test.shape}")

Train set shape : (4736,)
Test set shape : (836,)


In [23]:
#tokenizer 학습시키기
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)

In [24]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8210


In [26]:
# 2. pad_sequence로 패딩 처리를 하여 길이가 맞지 않는 배열의 형태를 통일시켜줌.
X_train_encoded = tokenizer.texts_to_sequences(X_train)

max_len = max(len(sent) for sent in X_train_encoded)
print(max_len)

96


In [28]:
#maxlen = 150으로 설정해주어야함.
X_train=pad_sequences(X_train_encoded, maxlen=150, padding='post')
y_train=np.array(y_train)

In [30]:
#word2vec의 임베딩 가중치 행렬 만들어주기
#vocab에 속하는 단어들에 대해서 만들어주기

#임베딩 백터 차원을 지정하여 만들어준다.
embedding_matrix = np.zeros((vocab_size, 300))

print(np.shape(embedding_matrix))

(8210, 300)


In [31]:
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [32]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.22558594, -0.01953125,  0.09082031, ...,  0.02819824,
        -0.17773438, -0.00604248],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.01867676,  0.24023438,  0.12402344, ...,  0.09716797,
        -0.13476562,  0.30078125],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [34]:
# 인공 신경망 모델링

#기본 모델 만들기
model = Sequential()
model.add(Embedding(vocab_size,300, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(GlobalAveragePooling1D())#입력되는 단어 벡터의 평균을 구하는 함수
model.add(Dense(1, activation='sigmoid'))

In [35]:
#모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
60/60 [==============================] - 2s 12ms/step - loss: 0.6765 - acc: 0.8229 - val_loss: 0.6566 - val_acc: 0.8660
Epoch 2/10
60/60 [==============================] - 1s 9ms/step - loss: 0.6404 - acc: 0.8648 - val_loss: 0.6225 - val_acc: 0.8660
Epoch 3/10
60/60 [==============================] - 1s 9ms/step - loss: 0.6088 - acc: 0.8648 - val_loss: 0.5930 - val_acc: 0.8660
Epoch 4/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5818 - acc: 0.8648 - val_loss: 0.5671 - val_acc: 0.8660
Epoch 5/10
60/60 [==============================] - 1s 10ms/step - loss: 0.5584 - acc: 0.8648 - val_loss: 0.5452 - val_acc: 0.8660
Epoch 6/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5382 - acc: 0.8648 - val_loss: 0.5263 - val_acc: 0.8660
Epoch 7/10
60/60 [==============================] - 1s 9ms/step - loss: 0.5207 - acc: 0.8648 - val_loss: 0.5095 - val_acc: 0.8660
Epoch 8/10
60/60 [==============================] - 1s 8ms/step - loss: 0.5056 - acc: 0.

In [36]:
#성능 평가하기

#테스트 데이터 인코딩하기
X_test_encoded = tokenizer.texts_to_sequences(X_test)
X_test=pad_sequences(X_test_encoded, maxlen=maxlen, padding='post')
y_test=np.array(y_test)

In [37]:
#모델 성능 평가하기
model.evaluate(X_test, y_test)

27/27 [==============================] - 0s 5ms/step - loss: 0.4712 - acc: 0.8708


[0.47120505571365356, 0.8708133697509766]